In [284]:
import pandas as pd 
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

In [285]:
def count_encoder(train,test,features:list):
    for feature in features:
        for df in (train,test):
            df[feature] = df[feature].map(df[feature].value_counts(normalize=True))


def agg_count(train,test,features:list,aggregation_feature:str,func):
    for feature in features:
        for df in (train,test):
            df[feature+'agg_count'] = df[feature].groupby(aggregation_feature).agg(func)


def mode (train,test,feature,aggregation_feature):
    for df in (train,test):
        df[feature+'_mode_'+aggregation_feature] = df[aggregation_feature].map(df.group_by(aggregation_feature)[feature].mode())


def stats_with_features(train,test,func,*feat):
    for df in (train,test):
        name = ''
        for f in feat:
            name += f
        df[name+func] = df[list(feat)].agg(func,axis=1)

def stats_by_feat(train,test,feat_to_stat,feat_to_group,func):
    for df in (train,test):
        df[feat_to_stat+feat_to_group+func] = df[feat_to_group].map(df.groupby(feat_to_group)[feat_to_stat].aggregate(func))

def to_date(train,test,*feats):
    for df in (train,test):
        for feature in feats:
            df[feature] = pd.to_datetime(df[feature])

def date(train,test,date):
    for df in (train,test):
        df[date+'day'] = df[date].dt.dayofyear
        df[date+'month'] = df[date].dt.month
        df[date+'year'] = df[date].dt.year

In [286]:
pd.options.display.max_columns=69

In [287]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [288]:
ID = test['Student ID']

for df in (train,test):
    df.drop(columns=['Student ID','Program ID'],inplace=True)

In [289]:
train

,Age,Gender,Home Region,Home City,Program Main Category Code,Program Sub Category Code,Technology Type,Program Skill Level,Program Presentation Method,Program Start Date,Program End Date,Program Days,Completed Degree,Level of Education,Education Speaciality,College,University Degree Score,University Degree Score System,Employment Status,Job Type,Still Working,Y
0,37.0,ذكر,منطقة الرياض,الرياض,PCRF,PCRF,NaN,NaN,حضوري,2023-05-28,2023-06-08,12,نعم,البكالوريوس,هندسة حاسب الالي,NaN,2.44,4.0,غير موظف,NaN,NaN,0
1,21.0,ذكر,منطقة عسير,خميس مشيط,APMR,SWPS,NaN,متوسط,حضوري,2023-04-02,2023-04-06,5,نعم,البكالوريوس,الإذاعة والتلفزيون والفيلم,الفنون والعلوم الإنسانية,5.00,5.0,طالب,NaN,NaN,0
2,24.0,ذكر,منطقة الرياض,الرياض,APMR,NaN,NaN,متوسط,حضوري,2023-07-23,2023-09-14,54,نعم,البكالوريوس,Information Technology,NaN,3.50,5.0,موظف,NaN,NaN,0
3,23.0,ذكر,منطقة الرياض,الرياض,TOSL,TOSL,NaN,NaN,حضوري,2023-07-23,2023-08-24,33,نعم,البكالوريوس,حوسبة تطبيقية - (مسار شبكات الحاسب),NaN,3.55,5.0,خريج,NaN,NaN,0
4,23.0,ذكر,منطقة الرياض,الرياض,CAUF,SWPS,تقليدية,متوسط,حضوري,2023-04-30,2023-06-22,54,لا,البكالوريوس,نظم المعلومات الحاسوبية,تكنولوجيا الاتصالات والمعلومات,4.00,5.0,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6543,31.0,أنثى,منطقة الرياض,الرياض,CAUF,SWPS,NaN,NaN,حضوري,2023-04-02,2023-06-06,66,نعم,البكالوريوس,تقنية المعلومات,تكنولوجيا الاتصالات والمعلومات,4.40,5.0,NaN,NaN,NaN,0
6544,27.0,أنثى,منطقة القصيم,بريدة,PCRF,PCRF,NaN,مبتدئ,عن بعد,2023-05-14,2023-05-25,12,نعم,البكالوريوس,علوم الحاسب,NaN,4.46,5.0,موظف,NaN,NaN,0
6545,24.0,أنثى,منطقة الرياض,الرياض,PCRF,PCRF,تقليدية,مبتدئ,حضوري,2023-07-16,2023-07-20,5,لا,البكالوريوس,نظم المعلومات,تكنولوجيا الاتصالات والمعلومات,4.93,5.0,موظف,دوام كامل,Yes,1
6546,25.0,ذكر,منطقة الرياض,الرياض,PCRF,PCRF,NaN,متوسط,حضوري,2024-01-07,2024-01-18,12,نعم,البكالوريوس,تقنية المعلومات,تكنولوجيا الاتصالات والمعلومات,4.00,4.0,غير موظف,تدريب,No,0


In [290]:
test

,Age,Gender,Home Region,Home City,Program Main Category Code,Program Sub Category Code,Technology Type,Program Skill Level,Program Presentation Method,Program Start Date,Program End Date,Program Days,Completed Degree,Level of Education,Education Speaciality,College,University Degree Score,University Degree Score System,Employment Status,Job Type,Still Working
0,23.0,أنثى,منطقة الرياض,الرياض,CAUF,SWPS,تقليدية,متوسط,عن بعد,2023-10-08,2023-10-12,5,لا,البكالوريوس,علوم الحاسب الالي,NaN,3.72,4.0,خريج,NaN,NaN
1,31.0,أنثى,منطقة الرياض,الرياض,PCRF,PCRF,تقليدية,مبتدئ,عن بعد,2023-07-16,2023-08-03,19,لا,البكالوريوس,تقنية المعلومات,تكنولوجيا الاتصالات والمعلومات,2.00,4.0,NaN,NaN,NaN
2,29.0,أنثى,منطقة الرياض,الرياض,CAUF,SWPS,NaN,NaN,عن بعد,2022-12-25,2023-01-05,12,نعم,البكالوريوس,شبكات الحاسب الآلي,تكنولوجيا الاتصالات والمعلومات,3.72,5.0,موظف,دوام كامل,Yes
3,23.0,أنثى,منطقة الرياض,الرياض,PCRF,PCRF,NaN,متقدم,حضوري,2023-03-19,2023-03-23,5,نعم,البكالوريوس,Biology,NaN,4.47,5.0,غير موظف,NaN,NaN
4,30.0,أنثى,منطقة الرياض,الرياض,TOSL,NaN,داعمة,متقدم,حضوري,2023-11-12,2023-12-14,33,نعم,الدبلوم,تقنية المعلومات,تكنولوجيا الاتصالات والمعلومات,4.46,5.0,غير موظف,دوام كامل,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
813,36.0,ذكر,منطقة الرياض,الرياض,GRST,INFA,NaN,متوسط,حضوري,2023-08-13,2023-08-17,5,نعم,البكالوريوس,علوم حاسب,NaN,2.55,5.0,موظف,NaN,NaN
814,29.0,ذكر,منطقة الرياض,الرياض,CAUF,CRDP,داعمة,مبتدئ,حضوري,2023-11-26,2024-01-11,47,نعم,البكالوريوس,الهندسة الكهربائية,الهندسة والتصنيع والبناء,3.00,4.0,NaN,NaN,NaN
815,32.0,ذكر,منطقة الرياض,الرياض,GRST,INFA,تقليدية,متوسط,حضوري,2023-07-23,2023-07-27,5,نعم,البكالوريوس,شبكات حاسب,NaN,3.00,5.0,موظف,NaN,NaN
816,28.0,أنثى,منطقة الرياض,الرياض,PCRF,PCRF,تقليدية,متوسط,عن بعد,2023-05-14,2023-11-02,173,لا,البكالوريوس,نظم معلومات حاسوبية,NaN,4.12,5.0,غير موظف,NaN,NaN


In [291]:
for df in (train,test):
    df['Gender']=df['Gender'].map({'أنثى':0,'ذكر':1})

In [292]:

cat_cols = list(train.select_dtypes(exclude='number').columns)
for col in cat_cols:
    print(col,train[col].nunique())

Home Region 13
Home City 92
Program Main Category Code 10
Program Sub Category Code 11
Technology Type 3
Program Skill Level 3
Program Presentation Method 2
Program Start Date 77
Program End Date 88
Completed Degree 2
Level of Education 5
Education Speaciality 871
College 9
Employment Status 6
Job Type 4
Still Working 2


In [293]:
train['Education Speaciality']

0                          هندسة حاسب الالي
1                الإذاعة والتلفزيون والفيلم
2                    Information Technology
3       حوسبة تطبيقية - (مسار شبكات الحاسب)
4                   نظم المعلومات الحاسوبية
                       ...                 
6543                        تقنية المعلومات
6544                            علوم الحاسب
6545                          نظم المعلومات
6546                        تقنية المعلومات
6547                   دراسات وقضايا معاصرة
Name: Education Speaciality, Length: 6548, dtype: object

In [294]:
train

,Age,Gender,Home Region,Home City,Program Main Category Code,Program Sub Category Code,Technology Type,Program Skill Level,Program Presentation Method,Program Start Date,Program End Date,Program Days,Completed Degree,Level of Education,Education Speaciality,College,University Degree Score,University Degree Score System,Employment Status,Job Type,Still Working,Y
0,37.0,1,منطقة الرياض,الرياض,PCRF,PCRF,NaN,NaN,حضوري,2023-05-28,2023-06-08,12,نعم,البكالوريوس,هندسة حاسب الالي,NaN,2.44,4.0,غير موظف,NaN,NaN,0
1,21.0,1,منطقة عسير,خميس مشيط,APMR,SWPS,NaN,متوسط,حضوري,2023-04-02,2023-04-06,5,نعم,البكالوريوس,الإذاعة والتلفزيون والفيلم,الفنون والعلوم الإنسانية,5.00,5.0,طالب,NaN,NaN,0
2,24.0,1,منطقة الرياض,الرياض,APMR,NaN,NaN,متوسط,حضوري,2023-07-23,2023-09-14,54,نعم,البكالوريوس,Information Technology,NaN,3.50,5.0,موظف,NaN,NaN,0
3,23.0,1,منطقة الرياض,الرياض,TOSL,TOSL,NaN,NaN,حضوري,2023-07-23,2023-08-24,33,نعم,البكالوريوس,حوسبة تطبيقية - (مسار شبكات الحاسب),NaN,3.55,5.0,خريج,NaN,NaN,0
4,23.0,1,منطقة الرياض,الرياض,CAUF,SWPS,تقليدية,متوسط,حضوري,2023-04-30,2023-06-22,54,لا,البكالوريوس,نظم المعلومات الحاسوبية,تكنولوجيا الاتصالات والمعلومات,4.00,5.0,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6543,31.0,0,منطقة الرياض,الرياض,CAUF,SWPS,NaN,NaN,حضوري,2023-04-02,2023-06-06,66,نعم,البكالوريوس,تقنية المعلومات,تكنولوجيا الاتصالات والمعلومات,4.40,5.0,NaN,NaN,NaN,0
6544,27.0,0,منطقة القصيم,بريدة,PCRF,PCRF,NaN,مبتدئ,عن بعد,2023-05-14,2023-05-25,12,نعم,البكالوريوس,علوم الحاسب,NaN,4.46,5.0,موظف,NaN,NaN,0
6545,24.0,0,منطقة الرياض,الرياض,PCRF,PCRF,تقليدية,مبتدئ,حضوري,2023-07-16,2023-07-20,5,لا,البكالوريوس,نظم المعلومات,تكنولوجيا الاتصالات والمعلومات,4.93,5.0,موظف,دوام كامل,Yes,1
6546,25.0,1,منطقة الرياض,الرياض,PCRF,PCRF,NaN,متوسط,حضوري,2024-01-07,2024-01-18,12,نعم,البكالوريوس,تقنية المعلومات,تكنولوجيا الاتصالات والمعلومات,4.00,4.0,غير موظف,تدريب,No,0


# Feature Engineering

In [295]:
to_date(train,test,'Program Start Date')
to_date(train,test,'Program End Date')

In [296]:
date(train,test,'Program Start Date')
date(train,test,'Program End Date')

In [297]:
for df in (train,test):
    df.drop(columns=['Program Start Date','Program End Date','Program Start Dateyear','Program End Dateyear'],inplace=True)

In [298]:
train

,Age,Gender,Home Region,Home City,Program Main Category Code,Program Sub Category Code,Technology Type,Program Skill Level,Program Presentation Method,Program Days,Completed Degree,Level of Education,Education Speaciality,College,University Degree Score,University Degree Score System,Employment Status,Job Type,Still Working,Y,Program Start Dateday,Program Start Datemonth,Program End Dateday,Program End Datemonth
0,37.0,1,منطقة الرياض,الرياض,PCRF,PCRF,NaN,NaN,حضوري,12,نعم,البكالوريوس,هندسة حاسب الالي,NaN,2.44,4.0,غير موظف,NaN,NaN,0,148,5,159,6
1,21.0,1,منطقة عسير,خميس مشيط,APMR,SWPS,NaN,متوسط,حضوري,5,نعم,البكالوريوس,الإذاعة والتلفزيون والفيلم,الفنون والعلوم الإنسانية,5.00,5.0,طالب,NaN,NaN,0,92,4,96,4
2,24.0,1,منطقة الرياض,الرياض,APMR,NaN,NaN,متوسط,حضوري,54,نعم,البكالوريوس,Information Technology,NaN,3.50,5.0,موظف,NaN,NaN,0,204,7,257,9
3,23.0,1,منطقة الرياض,الرياض,TOSL,TOSL,NaN,NaN,حضوري,33,نعم,البكالوريوس,حوسبة تطبيقية - (مسار شبكات الحاسب),NaN,3.55,5.0,خريج,NaN,NaN,0,204,7,236,8
4,23.0,1,منطقة الرياض,الرياض,CAUF,SWPS,تقليدية,متوسط,حضوري,54,لا,البكالوريوس,نظم المعلومات الحاسوبية,تكنولوجيا الاتصالات والمعلومات,4.00,5.0,NaN,NaN,NaN,0,120,4,173,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6543,31.0,0,منطقة الرياض,الرياض,CAUF,SWPS,NaN,NaN,حضوري,66,نعم,البكالوريوس,تقنية المعلومات,تكنولوجيا الاتصالات والمعلومات,4.40,5.0,NaN,NaN,NaN,0,92,4,157,6
6544,27.0,0,منطقة القصيم,بريدة,PCRF,PCRF,NaN,مبتدئ,عن بعد,12,نعم,البكالوريوس,علوم الحاسب,NaN,4.46,5.0,موظف,NaN,NaN,0,134,5,145,5
6545,24.0,0,منطقة الرياض,الرياض,PCRF,PCRF,تقليدية,مبتدئ,حضوري,5,لا,البكالوريوس,نظم المعلومات,تكنولوجيا الاتصالات والمعلومات,4.93,5.0,موظف,دوام كامل,Yes,1,197,7,201,7
6546,25.0,1,منطقة الرياض,الرياض,PCRF,PCRF,NaN,متوسط,حضوري,12,نعم,البكالوريوس,تقنية المعلومات,تكنولوجيا الاتصالات والمعلومات,4.00,4.0,غير موظف,تدريب,No,0,7,1,18,1


In [299]:
stats_by_feat(train,test,'University Degree Score','Home City','mean')


In [300]:
traintest = pd.concat([train,test])

In [301]:
traintest = pd.get_dummies(traintest)

In [302]:
train = traintest[:len(train)]
test = traintest[len(train):]

In [303]:
test

,Age,Gender,Program Days,University Degree Score,University Degree Score System,Y,Program Start Dateday,Program Start Datemonth,Program End Dateday,Program End Datemonth,University Degree ScoreHome Citymean,Home Region_المنطقة الشرقية,Home Region_منطقة الباحة,Home Region_منطقة الجوف,Home Region_منطقة الحدود الشمالية,Home Region_منطقة الرياض,Home Region_منطقة القصيم,Home Region_منطقة المدينة المنورة,Home Region_منطقة تبوك,Home Region_منطقة جازان,Home Region_منطقة حائل,Home Region_منطقة عسير,Home Region_منطقة مكة المكرمة,Home Region_منطقة نجران,Home City_أبها,Home City_أبو عريش,Home City_أحد المسارحة,Home City_أحد رفيدة,Home City_أملج,Home City_ابها,Home City_الأحساء,Home City_الباحة,Home City_البدائع,Home City_البكيرية,...,Education Speaciality_هندسة نظم معلومات,Education Speaciality_هندسة وأمن الشبكات,Education Speaciality_هندسه برمجيات,Education Speaciality_هندسه حاسب,Education Speaciality_هندسه حاسب الالي,Education Speaciality_هندسه حاسب وشبكات,Education Speaciality_هندسه شبكات,Education Speaciality_هندسه شبكات واتصلات,Education Speaciality_هندسه صناعيه,Education Speaciality_هندسه كهربائية,Education Speaciality_هندسه ميكانيكية,Education Speaciality_هياكل ومحركات,Education Speaciality_وسائط متعددة تفاعلية,College_الأعمال والإدارة والقانون,College_البرامج والمؤهلات العامة,College_التعليم,College_الصحة والرفاة,College_العلوم الاجتماعية والصحافة والإعلام,College_العلوم الطبيعية والرياضيات والإحصاء,College_الفنون والعلوم الإنسانية,College_الهندسة والتصنيع والبناء,College_تكنولوجيا الاتصالات والمعلومات,Employment Status_خريج,Employment Status_طالب,Employment Status_عمل حر,Employment Status_غير موظف,Employment Status_موظف,Employment Status_موظف - طالب,Job Type_تدريب,Job Type_تطوع,Job Type_دوام جزئي,Job Type_دوام كامل,Still Working_No,Still Working_Yes
0,23.0,0,5,3.72,4.0,NaN,281,10,285,10,9.476438,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
1,31.0,0,19,2.00,4.0,NaN,197,7,215,8,9.476438,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
2,29.0,0,12,3.72,5.0,NaN,359,12,5,1,9.476438,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,True
3,23.0,0,5,4.47,5.0,NaN,78,3,82,3,9.476438,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
4,30.0,0,33,4.46,5.0,NaN,316,11,348,12,9.476438,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [304]:
train

,Age,Gender,Program Days,University Degree Score,University Degree Score System,Y,Program Start Dateday,Program Start Datemonth,Program End Dateday,Program End Datemonth,University Degree ScoreHome Citymean,Home Region_المنطقة الشرقية,Home Region_منطقة الباحة,Home Region_منطقة الجوف,Home Region_منطقة الحدود الشمالية,Home Region_منطقة الرياض,Home Region_منطقة القصيم,Home Region_منطقة المدينة المنورة,Home Region_منطقة تبوك,Home Region_منطقة جازان,Home Region_منطقة حائل,Home Region_منطقة عسير,Home Region_منطقة مكة المكرمة,Home Region_منطقة نجران,Home City_أبها,Home City_أبو عريش,Home City_أحد المسارحة,Home City_أحد رفيدة,Home City_أملج,Home City_ابها,Home City_الأحساء,Home City_الباحة,Home City_البدائع,Home City_البكيرية,...,Education Speaciality_هندسة نظم معلومات,Education Speaciality_هندسة وأمن الشبكات,Education Speaciality_هندسه برمجيات,Education Speaciality_هندسه حاسب,Education Speaciality_هندسه حاسب الالي,Education Speaciality_هندسه حاسب وشبكات,Education Speaciality_هندسه شبكات,Education Speaciality_هندسه شبكات واتصلات,Education Speaciality_هندسه صناعيه,Education Speaciality_هندسه كهربائية,Education Speaciality_هندسه ميكانيكية,Education Speaciality_هياكل ومحركات,Education Speaciality_وسائط متعددة تفاعلية,College_الأعمال والإدارة والقانون,College_البرامج والمؤهلات العامة,College_التعليم,College_الصحة والرفاة,College_العلوم الاجتماعية والصحافة والإعلام,College_العلوم الطبيعية والرياضيات والإحصاء,College_الفنون والعلوم الإنسانية,College_الهندسة والتصنيع والبناء,College_تكنولوجيا الاتصالات والمعلومات,Employment Status_خريج,Employment Status_طالب,Employment Status_عمل حر,Employment Status_غير موظف,Employment Status_موظف,Employment Status_موظف - طالب,Job Type_تدريب,Job Type_تطوع,Job Type_دوام جزئي,Job Type_دوام كامل,Still Working_No,Still Working_Yes
0,37.0,1,12,2.44,4.0,0.0,148,5,159,6,8.744391,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
1,21.0,1,5,5.00,5.0,0.0,92,4,96,4,15.447500,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False
2,24.0,1,54,3.50,5.0,0.0,204,7,257,9,8.744391,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
3,23.0,1,33,3.55,5.0,0.0,204,7,236,8,8.744391,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
4,23.0,1,54,4.00,5.0,0.0,120,4,173,6,8.744391,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

# EDA

In [305]:
train.describe()

,Age,Gender,Program Days,University Degree Score,University Degree Score System,Y,Program Start Dateday,Program Start Datemonth,Program End Dateday,Program End Datemonth,University Degree ScoreHome Citymean
count,6456.000000,6548.000000,6548.000000,6467.000000,6467.000000,6548.000000,6548.000000,6548.000000,6548.000000,6548.000000,6545.000000
mean,26.831165,0.440440,19.691662,8.224432,9.773929,0.158674,181.399969,6.509316,187.382407,6.712126,8.228084
std,5.535967,0.496478,32.112061,19.120384,21.259962,0.365400,108.715441,3.544787,109.524922,3.530927,2.458955
min,18.000000,0.000000,3.000000,0.000000,4.000000,0.000000,1.000000,1.000000,4.000000,1.000000,0.000000
25%,23.000000,0.000000,5.000000,3.300000,5.000000,0.000000,85.000000,3.000000,82.000000,3.000000,8.744391
50%,25.000000,0.000000,12.000000,4.000000,5.000000,0.000000,197.000000,7.000000,201.000000,7.000000,8.744391
75%,29.000000,1.000000,19.000000,4.510000,5.000000,0.000000,281.000000,10.000000,285.000000,10.000000,8.744391
max,57.000000,1.000000,292.000000,100.000000,100.000000,1.000000,365.000000,12.000000,363.000000,12.000000,43.144286


In [306]:
train['Y'].value_counts()

Y
0.0    5509
1.0    1039
Name: count, dtype: int64

In [307]:
train

,Age,Gender,Program Days,University Degree Score,University Degree Score System,Y,Program Start Dateday,Program Start Datemonth,Program End Dateday,Program End Datemonth,University Degree ScoreHome Citymean,Home Region_المنطقة الشرقية,Home Region_منطقة الباحة,Home Region_منطقة الجوف,Home Region_منطقة الحدود الشمالية,Home Region_منطقة الرياض,Home Region_منطقة القصيم,Home Region_منطقة المدينة المنورة,Home Region_منطقة تبوك,Home Region_منطقة جازان,Home Region_منطقة حائل,Home Region_منطقة عسير,Home Region_منطقة مكة المكرمة,Home Region_منطقة نجران,Home City_أبها,Home City_أبو عريش,Home City_أحد المسارحة,Home City_أحد رفيدة,Home City_أملج,Home City_ابها,Home City_الأحساء,Home City_الباحة,Home City_البدائع,Home City_البكيرية,...,Education Speaciality_هندسة نظم معلومات,Education Speaciality_هندسة وأمن الشبكات,Education Speaciality_هندسه برمجيات,Education Speaciality_هندسه حاسب,Education Speaciality_هندسه حاسب الالي,Education Speaciality_هندسه حاسب وشبكات,Education Speaciality_هندسه شبكات,Education Speaciality_هندسه شبكات واتصلات,Education Speaciality_هندسه صناعيه,Education Speaciality_هندسه كهربائية,Education Speaciality_هندسه ميكانيكية,Education Speaciality_هياكل ومحركات,Education Speaciality_وسائط متعددة تفاعلية,College_الأعمال والإدارة والقانون,College_البرامج والمؤهلات العامة,College_التعليم,College_الصحة والرفاة,College_العلوم الاجتماعية والصحافة والإعلام,College_العلوم الطبيعية والرياضيات والإحصاء,College_الفنون والعلوم الإنسانية,College_الهندسة والتصنيع والبناء,College_تكنولوجيا الاتصالات والمعلومات,Employment Status_خريج,Employment Status_طالب,Employment Status_عمل حر,Employment Status_غير موظف,Employment Status_موظف,Employment Status_موظف - طالب,Job Type_تدريب,Job Type_تطوع,Job Type_دوام جزئي,Job Type_دوام كامل,Still Working_No,Still Working_Yes
0,37.0,1,12,2.44,4.0,0.0,148,5,159,6,8.744391,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
1,21.0,1,5,5.00,5.0,0.0,92,4,96,4,15.447500,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False
2,24.0,1,54,3.50,5.0,0.0,204,7,257,9,8.744391,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
3,23.0,1,33,3.55,5.0,0.0,204,7,236,8,8.744391,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
4,23.0,1,54,4.00,5.0,0.0,120,4,173,6,8.744391,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [308]:
test

,Age,Gender,Program Days,University Degree Score,University Degree Score System,Y,Program Start Dateday,Program Start Datemonth,Program End Dateday,Program End Datemonth,University Degree ScoreHome Citymean,Home Region_المنطقة الشرقية,Home Region_منطقة الباحة,Home Region_منطقة الجوف,Home Region_منطقة الحدود الشمالية,Home Region_منطقة الرياض,Home Region_منطقة القصيم,Home Region_منطقة المدينة المنورة,Home Region_منطقة تبوك,Home Region_منطقة جازان,Home Region_منطقة حائل,Home Region_منطقة عسير,Home Region_منطقة مكة المكرمة,Home Region_منطقة نجران,Home City_أبها,Home City_أبو عريش,Home City_أحد المسارحة,Home City_أحد رفيدة,Home City_أملج,Home City_ابها,Home City_الأحساء,Home City_الباحة,Home City_البدائع,Home City_البكيرية,...,Education Speaciality_هندسة نظم معلومات,Education Speaciality_هندسة وأمن الشبكات,Education Speaciality_هندسه برمجيات,Education Speaciality_هندسه حاسب,Education Speaciality_هندسه حاسب الالي,Education Speaciality_هندسه حاسب وشبكات,Education Speaciality_هندسه شبكات,Education Speaciality_هندسه شبكات واتصلات,Education Speaciality_هندسه صناعيه,Education Speaciality_هندسه كهربائية,Education Speaciality_هندسه ميكانيكية,Education Speaciality_هياكل ومحركات,Education Speaciality_وسائط متعددة تفاعلية,College_الأعمال والإدارة والقانون,College_البرامج والمؤهلات العامة,College_التعليم,College_الصحة والرفاة,College_العلوم الاجتماعية والصحافة والإعلام,College_العلوم الطبيعية والرياضيات والإحصاء,College_الفنون والعلوم الإنسانية,College_الهندسة والتصنيع والبناء,College_تكنولوجيا الاتصالات والمعلومات,Employment Status_خريج,Employment Status_طالب,Employment Status_عمل حر,Employment Status_غير موظف,Employment Status_موظف,Employment Status_موظف - طالب,Job Type_تدريب,Job Type_تطوع,Job Type_دوام جزئي,Job Type_دوام كامل,Still Working_No,Still Working_Yes
0,23.0,0,5,3.72,4.0,NaN,281,10,285,10,9.476438,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
1,31.0,0,19,2.00,4.0,NaN,197,7,215,8,9.476438,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
2,29.0,0,12,3.72,5.0,NaN,359,12,5,1,9.476438,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,True
3,23.0,0,5,4.47,5.0,NaN,78,3,82,3,9.476438,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
4,30.0,0,33,4.46,5.0,NaN,316,11,348,12,9.476438,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


# Validation

In [309]:
lgbm = LGBMClassifier(random_state=42)


In [310]:
stratfiedkfold = StratifiedKFold(shuffle=True,random_state=69)
f1 = []
for v_train,v_test in stratfiedkfold.split(train.drop(columns='Y'),train['Y']):
    train_v, test_V= train.iloc[v_train],train.iloc[v_test]
    lgbm.fit(np.array(train_v.drop(columns='Y')),train_v['Y'])
    pred = lgbm.predict(np.array(test_V.drop(columns='Y')))
    score = f1_score(pred,test_V['Y'])
    print(score)
    f1.append(score)
            

[LightGBM] [Info] Number of positive: 831, number of negative: 4407
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002072 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 786
[LightGBM] [Info] Number of data points in the train set: 5238, number of used features: 121
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.158648 -> initscore=-1.668320
[LightGBM] [Info] Start training from score -1.668320
0.6187845303867403
[LightGBM] [Info] Number of positive: 831, number of negative: 4407
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001520 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 795
[LightGBM] [Info] Number of data points in the train set: 5238, number of used features: 122
[

In [311]:
np.array(f1).mean()

0.5917566419342132

In [312]:
lgbm.fit(np.array(train.drop(columns='Y')),train['Y'])

[LightGBM] [Info] Number of positive: 1039, number of negative: 5509
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001786 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 812
[LightGBM] [Info] Number of data points in the train set: 6548, number of used features: 130
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.158674 -> initscore=-1.668124
[LightGBM] [Info] Start training from score -1.668124


LGBMClassifier(random_state=42)

In [313]:
pred = lgbm.predict(np.array(test.drop(columns='Y')))

In [314]:
sub = pd.DataFrame({
    'ID':range(1,len(pred)+1),
    'Y':pred
})

In [315]:
sub.to_csv('submission.csv',index=False)